In [3]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from SUNRGBDLoader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', 
                    help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, 
                    help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, 
                    help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', 
                    help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', 
                    help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=100, 
                    help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, 
                    help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, 
                    help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, 
                    help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None,    
                    help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', 
                    help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', 
                    help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)


In [5]:
args = parser.parse_args('--arch fcn8s_with_rgbd --batch_size 3'.split(' '))

# Setup Augmentations
data_aug= Compose([RandomRotate(10),                                        
                   RandomHorizontallyFlip()])

# Setup Dataloader
data_path = '/home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD(renamed)/'
t_loader = SUNRGBDLoader(data_path, is_transform=True)
v_loader = SUNRGBDLoader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


# Training

In [ ]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        images = Variable(color_imgs.cuda())
        depth_images = Variable(depth_imgs.cuda())
        labels = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(images,depth_images)

        loss = loss_fn(input=outputs, target=labels)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 100 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val, depth_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "{}_{}_best_model.pkl".format(args.arch, 'SUNRGBD'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/100] Loss: 2.5653
Epoch [1/100] Loss: 2.4659
Epoch [1/100] Loss: 2.0926
Epoch [1/100] Loss: 1.8335
Epoch [1/100] Loss: 1.8200
Epoch [1/100] Loss: 1.6252
Epoch [1/100] Loss: 1.7734
Epoch [1/100] Loss: 1.6339
Epoch [1/100] Loss: 1.5437
Epoch [1/100] Loss: 1.7697
Epoch [1/100] Loss: 1.3772
Epoch [1/100] Loss: 1.4250
Epoch [1/100] Loss: 1.3441
Epoch [1/100] Loss: 1.3537
Epoch [1/100] Loss: 1.4647
Epoch [1/100] Loss: 1.8065
Epoch [1/100] Loss: 1.7502


1684it [07:56,  3.53it/s]
/home/dongwonshin/Desktop/pytorch_dataloader/ptsemseg/metrics.py:32: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/home/dongwonshin/Desktop/pytorch_dataloader/ptsemseg/metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


Overall Acc: 	 0.50127780657
Mean Acc : 	 0.237929880159
FreqW Acc : 	 0.31817644532
Mean IoU : 	 0.147252524509
Epoch [2/100] Loss: 1.4518
Epoch [2/100] Loss: 1.3407
Epoch [2/100] Loss: 1.7499
Epoch [2/100] Loss: 1.3083
Epoch [2/100] Loss: 1.5426
Epoch [2/100] Loss: 1.4805
Epoch [2/100] Loss: 1.6745
Epoch [2/100] Loss: 1.8208
Epoch [2/100] Loss: 2.0018
Epoch [2/100] Loss: 1.5259
Epoch [2/100] Loss: 1.5382
Epoch [2/100] Loss: 1.2382
Epoch [2/100] Loss: 1.7711
Epoch [2/100] Loss: 1.3658
Epoch [2/100] Loss: 1.1755
Epoch [2/100] Loss: 1.2492
Epoch [2/100] Loss: 1.2845


1684it [07:56,  3.53it/s]


...

Epoch [41/100] Loss: 0.6316
Epoch [41/100] Loss: 0.7508
Epoch [41/100] Loss: 0.8783
Epoch [41/100] Loss: 0.6928
Epoch [41/100] Loss: 0.9295
Epoch [41/100] Loss: 0.8055
Epoch [41/100] Loss: 1.0065
Epoch [41/100] Loss: 0.7945
Epoch [41/100] Loss: 0.4616
Epoch [41/100] Loss: 0.9402
Epoch [41/100] Loss: 0.9037
Epoch [41/100] Loss: 0.5888
Epoch [41/100] Loss: 0.6541
Epoch [41/100] Loss: 0.7821
Epoch [41/100] Loss: 0.5742
Epoch [41/100] Loss: 0.8008
Epoch [41/100] Loss: 1.2732


1684it [07:58,  3.52it/s]

Overall Acc: 	 0.618035228445
Mean Acc : 	 0.484082206023
FreqW Acc : 	 0.45129514418
Mean IoU : 	 0.3485075682


Epoch [42/100] Loss: 1.1019
Epoch [42/100] Loss: 0.8609
Epoch [42/100] Loss: 1.0200
Epoch [42/100] Loss: 1.1195
Epoch [42/100] Loss: 1.1333
Epoch [42/100] Loss: 0.8483
Epoch [42/100] Loss: 0.7055
Epoch [42/100] Loss: 1.0333
Epoch [42/100] Loss: 0.5886
Epoch [42/100] Loss: 0.9491
Epoch [42/100] Loss: 0.9739
Epoch [42/100] Loss: 0.7142
Epoch [42/100] Loss: 0.5804
Epoch [42/100] Loss: 0.8288
Epoch [42/100] Loss: 0.8307
Epoch [42/100] Loss: 0.6795
Epoch [42/100] Loss: 0.4623


111it [00:32,  3.42it/s]